In [ ]:
val reportFolder = "reports"

In [ ]:
import java.io.File
import java.util.Properties

val properties = Properties()
File("gradle.properties").bufferedReader().use {
    properties.load(it)
}

val projectPath = properties.getProperty("project.path") ?: properties.getProperty("project.git.url")
val projectName: String = projectPath?.substringAfterLast("\\")?.substringAfterLast("/")?.removeSuffix(".git") ?: "User"

In [ ]:
enum class CompilerMetrics(val readableName: String) {
    GRADLE_TASK("Total Gradle task time"),

    COMPILATION_ROUND("Sources compilation round"),
    
    //compiler metrics
    COMPILER_INITIALIZATION("Compiler initialization time"),
    CODE_ANALYSIS("Compiler code analysis"),
    IR_TRANSLATION("Compiler IR translation"),
    IR_LOWERING("Compiler IR lowering"),
    IR_GENERATION("Compiler IR generation"),
    ;
}

In [ ]:
%use dataframe

In [ ]:
import java.io.File
import org.jetbrains.kotlinx.dataframe.DataFrame

//Json report print time in milliseconds 
fun readJsonReportFile(file: File): Map<CompilerMetrics, Long?> {
    val report = DataFrame.read(file)
    val timeMetrics = report.get("aggregatedMetrics").get("buildTimes").get("buildTimesNs")[0] as DataRow<*>

    return CompilerMetrics.values().associate {
        val value = timeMetrics.getOrNull(it.name)?.let { 
            when(it) {
                is Int -> it.toLong()
                is Long -> it
                is String -> it.toLong()
                else -> throw IllegalStateException("Unknown type ${it.javaClass} to convert to Long")
            }
        }
        Pair(it, value)
    }
}

In [ ]:
import kotlin.streams.toList

//Text report print time in seconds with 2 decemal digits, so there is an error during conversion to nanoSeconds
fun readTextReportFile(file: File): Map<CompilerMetrics, Long?> {
    val aggregatedTimeMetrics = file.bufferedReader().use {
        it.lines().dropWhile { !it.startsWith("Time metrics:") }.takeWhile { it.isNotBlank() }.toList()
    }

    return CompilerMetrics.values().associate { metric ->
        val metricLine = aggregatedTimeMetrics.firstOrNull { it.trim().startsWith(metric.readableName) }
        val value = metricLine?.substringAfter(": ")?.substringBefore(" ")?.replace(",", "")?.toDouble()
        Pair(metric, value?.times(1_000_000_000)?.toLong())
    }
}

# Read compiler metrics


In [ ]:
%use kandy

In [ ]:
import org.jetbrains.kotlinx.dataframe.api.toDataFrame
import java.nio.file.Files
import kotlin.io.path.Path

fun readMetrics(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>, predicate: (File) -> kotlin.Boolean
) = directory.listFiles().filter { !it.name.startsWith("buildSrc") }
    .filter(predicate)
    .sortedByDescending(File::lastModified)
    .toList().take(10).map {
         readReportFile(it)
    }.toList()

fun collectMeanValue(
    metrics: List<Map<CompilerMetrics, Long?>>,
): Map<String, Any?> {
    val rows = CompilerMetrics.values().map { compilerMetric ->
        val values = metrics.map { it[compilerMetric] }.filterNotNull()
        Pair(compilerMetric.name, values)
    }.filter { 
        val allValuesPresent = (it.second.size == 10)
        if (!allValuesPresent) {
            println("Unable to calculate mean value for ${it.first}, only ${it.second.size} runs are found")
        }
        allValuesPresent
    }.toTypedArray()

    val meanValue = dataFrameOf(*rows).mean()

    return CompilerMetrics.values().associate {
        Pair(it.name, meanValue.getOrNull(it.name))
    }
} 


# Collect data into dataframe

In [ ]:
val kotlinVersionColumn = "kotlinVersion"
val buildScenarioColumn = "buildScenario"

val columnNames = CompilerMetrics.values().map { it.name }.toMutableList()
columnNames.add(kotlinVersionColumn)
columnNames.add(buildScenarioColumn)

fun collectResultsIntoDataFrame(
    directory: File,
    readReportFile: (File) -> Map<CompilerMetrics, Long?>,
    predicate: (File) -> kotlin.Boolean,
): AnyFrame {
    val rows = directory.listFiles().flatMap { versionDir ->
        versionDir.listFiles().map { scenarioDir ->
            val buildMetrics = readMetrics(scenarioDir, readReportFile, predicate)
            //rename scenarios for graphics only
            val scenario = when (scenarioDir.name) {
                "clean_build" -> "Clean build"
                "incremental_abi_build" -> "Incremental build for ABI changes"
                "incremental_non_abi_build" -> "Incremental build for non ABI changes"
                else -> scenarioDir.name
            }

            println("\n Validate \'$scenario\' scenario for ${versionDir.name} kotlin version")
            val row = collectMeanValue(buildMetrics).toMutableMap()
            row.put(kotlinVersionColumn, versionDir.name)
            row.put(buildScenarioColumn, scenario)
            row
        }
    }

    val db = columnNames.map { compilerMetric -> Pair(compilerMetric, rows.map { it[compilerMetric] })}.toTypedArray()
    return dataFrameOf(*db)
}

In [ ]:

val dataFrame = collectResultsIntoDataFrame(File(reportFolder), ::readTextReportFile){
    it.extension == "txt" && it.readText().isNotEmpty()
}.sortBy(kotlinVersionColumn)
val scenarios = dataFrame[buildScenarioColumn]


In [ ]:
val plots = scenarios.distinct().toList().map { scenario ->
    dataFrame.filter { buildScenario == scenario }.plot {
        layout.title = "$projectName project - $scenario"
        bars {
            x(kotlinVersionColumn) {
                axis.name = "Kotlin version"
            }
            y(CompilerMetrics.COMPILATION_ROUND.name) {
                axis.name = "Compiler build time"
            }
        }
        
    }
}
plotGrid(plots, 1)

In [ ]:

fun gradleBuildTime(scenarios: List<String>, name: String) =
    dataFrame.filter { buildScenario in scenarios }.plot {
        layout.title = "$projectName project - $name"
        bars {
            x(buildScenarioColumn) {
                axis.name = "Gradle build time"
            }
            y(GRADLE_TASK.name) {
                axis.name = "Time in nanoseconds"
            }
            fillColor(kotlinVersionColumn) {
                legend.name = "Kotlin version"
            }
        }
    }


In [ ]:
gradleBuildTime(listOf("Clean build"), "Clean build")

In [ ]:
gradleBuildTime(listOf("Incremental build for ABI changes", "Incremental build for non ABI changes"), "Incremental build")

In [ ]:
fun kotlinCompilationTime(scenarios: List<String>, name: String) =
dataFrame.filter { buildScenario in scenarios }.plot {
    layout.title = "$projectName project - $name"
    bars {
        x(buildScenarioColumn) {
            axis.name = "Kotlin compilation time"
        }
        y(GRADLE_TASK.name) {
            axis.name = "Time in nanoseconds"
        }
        fillColor(kotlinVersionColumn) {
            legend.name = "Kotlin version"
        }
    }
}


In [ ]:
kotlinCompilationTime(listOf("Clean build"), "Clean build")


In [ ]:
kotlinCompilationTime(listOf("Incremental build for ABI changes", "Incremental build for non ABI changes"), "Incremental build")

# Compare Gradle and Compiler time

In [ ]:

 fun getPlotForGradleCompilerTime(scenario: String) = dataFrame.filter { it["buildScenario"] == scenario }
        .gather { GRADLE_TASK.name and COMPILATION_ROUND.name }.into("metric", "time_ns")
        .update("metric").with {
            when (it) {
                "GRADLE_TASK"  -> "Gradle time"
                "COMPILATION_ROUND" -> "Kotlin Compiler time"
                else -> throw IllegalStateException("Unknown metrics: $it")
            }
        }
        .sortBy("time_ns")
        .plot {
            layout {
                title = "$projectName project"
                subtitle = scenario
                xAxisLabel = "Gradle compilation breakdown"
                theme() {
                    legend.position = LegendPosition.Top
                }
            }
            barsH {
                y(kotlinVersion) {
                    axis.name = "Kotin version"
                }
                x("time_ns"<Double>()) {
                    axis.name = "Time in nanoseconds"
                }
                fillColor("metric") {
                    legend.name = "Build time"
                }
                position = Position.identity()
            }
        }




In [ ]:
val versionByScenario =
    scenarios.distinct().toList().associate { name -> Pair(name as String, getPlotForGradleCompilerTime(name)) }

In [ ]:
versionByScenario.get("Clean build")

In [ ]:
versionByScenario.get("Incremental build for non ABI changes")

In [ ]:
versionByScenario.get("Incremental build for ABI changes")

# Collect Compiler metrics

In [ ]:
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalContinuousScale
import org.jetbrains.kotlinx.kandy.ir.scale.PositionalScale

val metricsList = dataFrame
    .gather { COMPILER_INITIALIZATION.name and CODE_ANALYSIS.name and IR_TRANSLATION.name and IR_LOWERING.name and IR_GENERATION.name }.into("metric", "time_ns")


fun getPlotForCompilerMetrics(scenario: String) = metricsList.filter { it["buildScenario"] == scenario }
    .plot {
    layout {
        title = "$projectName project"
        subtitle = scenario
        xAxisLabel = "Kotlin compilation breakdown"
        theme() {
            legend.position = LegendPosition.Top
        }
        size = Pair(1000, 300)
    }
    barsH {
        y(kotlinVersion) {
            axis.name = "Kotlin version"
        }
        x("time_ns"<Double>()) {
            axis.name = "Time in nanoseconds"
        }
        fillColor("metric")
        position = Position.stack()
        
    }
}


In [ ]:
val compilerMetrics = scenarios.distinct().toList().associate { Pair(it as String, getPlotForCompilerMetrics(it)) }

In [ ]:
compilerMetrics.get("Clean build") 


In [ ]:
compilerMetrics.get("Incremental build for non ABI changes") 

In [ ]:
compilerMetrics.get("Incremental build for ABI changes") 